In [2]:
!pip install tensorflow-docs


Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 182.5/182.5 kB 1.9 MB/s eta 0:00:00


In [1]:
import keras

from keras import layers
from keras import ops
from tensorflow_docs.vis import embed
import tensorflow as tf
import numpy as np
import imageio

In [2]:
batch_size = 64
num_channels = 3
num_classes = 240
image_size = 64
latent_dim = 128

In [3]:
# Configure TensorFlow to use GPU for faster computation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
import pandas as pd
import numpy as np
import requests
from PIL import Image
from io import BytesIO
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.preprocessing.image import img_to_array

In [4]:
# Load the dataset
file_path = 'Segregated_Fashion_Dataset.csv'
data = pd.read_csv(file_path)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_4088\3101677453.py:3: DtypeWarning: Columns (0,13,27,35,36,38,40,41,42,65,69,72,73,74,75,76,79,80,81,82,83,89,90,91,92,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


In [5]:
# Select necessary columns (including 'img' for image URLs)
required_columns = ['brand','price', 'img','colour', 'Add-Ons', 'Body or Garment Size', 'Bottom Closure', 'Bottom Fabric', 'Bottom Pattern' , 'Bottom Type', 'Dupatta','Dupatta Border', 'Dupatta Fabric', 'Dupatta Pattern', 'Main Trend', 'Neck', 'Occasion', 'Pattern Coverage', 'Print or Pattern Type', 'Sleeve Length' , 'Sleeve Styling' , 'Stitch' , 'Sustainable', 'Top Design Styling' , 'Top Fabric' , 'Top Hemline' , 'Top Length' , 'Top Pattern' , 'Top Shape', 'Top Type' ]  # Adjust columns as per your dataset
data = data[required_columns]

# Take only the top 1000 rows
data = data.head(5000)
print(data.shape)

(5000, 30)


In [7]:
# Create directories for storing images
image_dir = 'fashion_images'
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

In [9]:
# Download and preprocess images, store them in a list
image_data = []
valid_indices = []

for i, url in enumerate(data['img']):
    if pd.notna(url):  # Check if URL is not NaN
        img_array = download_and_preprocess_image(url)
        if img_array is not None:
            image_data.append(img_array)
            valid_indices.append(i)  # Keep track of valid images
    else:
        print(f"Skipping row {i} due to missing image URL")

Skipping row 447 due to missing image URL
Skipping row 1447 due to missing image URL
Skipping row 1456 due to missing image URL
Skipping row 2378 due to missing image URL
Skipping row 2680 due to missing image URL


In [8]:
# Function to download and preprocess image
def download_and_preprocess_image(url, target_size=(64, 64)):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content)).convert('RGB')
            img = img.resize(target_size)
            img_array = img_to_array(img)
            img_array = img_array.astype("float32") / 255.0 #normalizing each value of pixel within the range 0 to 1
            return img_array
        else:
            print(f"Failed to download image from {url}: HTTP status code {response.status_code}")
            return None
    except Exception as e:
        print(f"Failed to download image from {url}: {e}")
        return None

In [6]:
# Filter data to only include rows with valid images


data = data.iloc[valid_indices].reset_index(drop=True)
image_data = np.array(image_data)

# One-hot encode categorical columns
categorical_columns = ['colour', 'Bottom Closure', 'Bottom Fabric', 'Bottom Pattern' , 'Bottom Type', 'Neck', 'Pattern Coverage', 'Print or Pattern Type', 'Sleeve Length' , 'Sleeve Styling' , 'Top Design Styling' , 'Top Fabric' , 'Top Hemline' , 'Top Length' , 'Top Pattern' , 'Top Shape', 'Top Type' ]  # Adjust columns as per your dataset

#Converting into a Pandas dataframe
df = pd.DataFrame(data)

# Handling missing values for all categorical columns
for col in categorical_columns:
    data[col].fillna('Unknown', inplace=True)  # Replace missing values with 'Unknown'
# Removing rows with missing values
df_cleaned = df.dropna()

#Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Apply one-hot encoding to the categorical columns
one_hot_encoded = encoder.fit_transform(df[categorical_columns])


#Create a DataFrame with the one-hot encoded columns
#We use get_feature_names_out() to get the column names for the encoded data
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Concatenate the one-hot encoded dataframe with the original dataframe
df_encoded = pd.concat([df, one_hot_df], axis=1)

# Drop the original categorical columns
df_encoded = df_encoded.drop(categorical_columns, axis=1)

df_encoded = df_encoded.drop('img', axis=1)
df_encoded = df_encoded.drop('brand', axis=1)
df_encoded = df_encoded.drop('Add-Ons', axis=1)
df_encoded = df_encoded.drop('Body or Garment Size', axis=1)
df_encoded = df_encoded.drop('Occasion', axis=1)
df_encoded = df_encoded.drop('Stitch', axis=1)
df_encoded = df_encoded.drop('Sustainable', axis=1)
df_encoded = df_encoded.drop('Dupatta', axis=1)
df_encoded = df_encoded.drop('Dupatta Border', axis=1)
df_encoded = df_encoded.drop('Dupatta Fabric', axis=1)
df_encoded = df_encoded.drop('Dupatta Pattern', axis=1)
df_encoded = df_encoded.drop('Main Trend', axis=1)

from sklearn.preprocessing import StandardScaler

# Handle missing values in the price column by filling with the median price
df_encoded['price'].fillna(df_encoded['price'].median(), inplace=True)

# Initialize the StandardScaler to normalize the price
scaler = StandardScaler()

# Reshape the price column to fit the scaler and scale it
df_encoded['price'] = scaler.fit_transform(df_encoded[['price']])

# Display the first few rows of the processed dataframe to confirm the changes
print(f"Processed data with scaled price: \n{df_encoded.head()}")


# Display the resulting dataframe
print(f"Encoded data : \n{df_encoded}")




NameError: name 'valid_indices' is not defined

In [7]:
print("Features shape:", df_encoded.shape)
print("Labels shape:", image_data.shape)


NameError: name 'df_encoded' is not defined

In [13]:
from sklearn.model_selection import train_test_split
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(image_data, df_encoded, test_size=0.3, random_state=42)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=1024).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

# Example of the shape
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_test: {y_test.shape}")

Shape of X_train: (3496, 64, 64, 3)
Shape of y_train: (3496, 240)
Shape of X_test: (1499, 64, 64, 3)
Shape of y_test: (1499, 240)


In [8]:
all_images = np.concatenate([X_train, X_test])
all_labels = np.concatenate([y_train, y_test])


NameError: name 'X_train' is not defined

In [15]:
# Create tf.data.Dataset.
dataset = tf.data.Dataset.from_tensor_slices((all_images, all_labels))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

print(f"Shape of training images: {all_images.shape}")
print(f"Shape of training labels: {all_labels.shape}")

Shape of training images: (4995, 64, 64, 3)
Shape of training labels: (4995, 240)


In [16]:
generator_in_channels = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes
print(generator_in_channels, discriminator_in_channels)

368 243


In [17]:
# Create the discriminator.
discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((64, 64, discriminator_in_channels)),  # Input shape is 64x64x3
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),  # Binary classification: Real (1) or Fake (0)
    ],
    name="discriminator",
)

# Create the generator.
generator = keras.Sequential(
    [
        keras.layers.InputLayer((128 + 240,)),  # Latent dimension + number of classes
        layers.Dense(8 * 8 * 256),
        layers.LeakyReLU(negative_slope=0.2),
        layers.Reshape((8, 8, 256)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.Conv2DTranspose(3, (4, 4), strides=(2, 2), padding="same", activation="tanh"),  # RGB output
    ],
    name="generator",
)

In [18]:
class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.seed_generator = keras.random.SeedGenerator(1337)
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data):
        # Unpack the data.
        real_images, one_hot_labels = data

        # Add dummy dimensions to the labels so that they can be concatenated with
        # the images. This is for the discriminator.
        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = ops.repeat(
            image_one_hot_labels, repeats=[image_size * image_size]
        )
        image_one_hot_labels = ops.reshape(
            image_one_hot_labels, (-1, image_size, image_size, num_classes)
        )

        # Sample random points in the latent space and concatenate the labels.
        # This is for the generator.
        batch_size = ops.shape(real_images)[0]
        random_latent_vectors = keras.random.normal(
            shape=(batch_size, self.latent_dim), seed=self.seed_generator
        )
        random_vector_labels = ops.concatenate(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Decode the noise (guided by labels) to fake images.
        generated_images = self.generator(random_vector_labels)

        # Combine them with real images. Note that we are concatenating the labels
        # with these images here.
        fake_image_and_labels = ops.concatenate(
            [generated_images, image_one_hot_labels], -1
        )
        real_image_and_labels = ops.concatenate([real_images, image_one_hot_labels], -1)
        combined_images = ops.concatenate(
            [fake_image_and_labels, real_image_and_labels], axis=0
        )

        # Assemble labels discriminating real from fake images.
        labels = ops.concatenate(
            [ops.ones((batch_size, 1)), ops.zeros((batch_size, 1))], axis=0
        )

        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space.
        random_latent_vectors = keras.random.normal(
            shape=(batch_size, self.latent_dim), seed=self.seed_generator
        )
        random_vector_labels = ops.concatenate(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = ops.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels)
            fake_image_and_labels = ops.concatenate(
                [fake_images, image_one_hot_labels], -1
            )
            predictions = self.discriminator(fake_image_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }

In [ ]:
cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

cond_gan.fit(dataset, epochs=100)

Epoch 1/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - d_loss: 0.3663 - g_loss: 3.6035
Epoch 2/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - d_loss: 0.0915 - g_loss: 7.6142
Epoch 3/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 153s 2s/step - d_loss: 0.0630 - g_loss: 7.9836
Epoch 4/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 154s 2s/step - d_loss: 0.0202 - g_loss: 9.1472
Epoch 5/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - d_loss: 0.2964 - g_loss: 5.4650
Epoch 6/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 155s 2s/step - d_loss: 0.0481 - g_loss: 4.3103
Epoch 7/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 149s 2s/step - d_loss: 0.0224 - g_loss: 5.0526
Epoch 8/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 149s 2s/step - d_loss: 0.2199 - g_loss: 9.1824
Epoch 9/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - d_loss: 0.0115 - g_loss: 7.9849
Epoch 10/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 148s 2s/step - d_loss: 0.0038 - g_loss: 8.1500
Epoch 11/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - d_loss: 0.0011 - g_loss: 8.9593
Epoch 12/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step

In [11]:
loss_fn=keras.losses.BinaryCrossentropy(from_logits=True)
loss_fn